In [ ]:
from __future__ import print_function

In [ ]:
import os
import json

In [ ]:
with open('./models/config.json', 'r') as f:
    config = json.load(f)
    
with open('./models/vocabulary/labels.txt', 'r') as f:
    vocab = {word: i for i, word in enumerate(f.readlines())}
    
serialization_dir = './models'
weights_file = './models/weights.th'

In [ ]:
from text_field_embedder import TextFieldEmbedder
from feedforward import FeedForward
from similarity_function import SimilarityFunction
from elmo_token_embedder import ElmoTokenEmbedder

In [ ]:
params = config.pop('model')
model_type = params.pop('type')
initializers = params.pop('initializer')

In [ ]:
embedder_params = params.pop('text_field_embedder')
token_embedders = {}
for key, emb_params in embedder_params.items():
    token_embedders[key] = ElmoTokenEmbedder(**emb_params)
text_field_embedder = TextFieldEmbedder(token_embedders)

In [ ]:
attend_ff_params = params.pop('attend_feedforward')
attend_feedforward = FeedForward(**attend_ff_params)

In [ ]:
similarity_func_params = params.pop('similarity_function')
similarity_function = SimilarityFunction()

In [ ]:
compare_ff_params = params.pop('compare_feedforward')
compare_feedforward = FeedForward(**compare_ff_params)

In [ ]:
aggregate_ff_params = params.pop('aggregate_feedforward')
aggregate_feedforward = FeedForward(**aggregate_ff_params)

In [ ]:
from model import DecomposableAttention

In [ ]:
model = DecomposableAttention(vocab, text_field_embedder, attend_feedforward,
                              similarity_function, compare_feedforward,
                              aggregate_feedforward)
model.load(serialization_dir, weights_file)

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
def tokenize(text):
    doc = nlp(text)
    return [token for token in doc]

In [ ]:
p = 'If you help the needy, God will reward you.'
h = 'Giving money to the poor has good consequences.'

In [ ]:
from elmo_indexer import ELMoTokenCharactersIndexer

In [ ]:
indexer = ELMoTokenCharactersIndexer()

In [ ]:
p_tokens = tokenize(p)
h_tokens = tokenize(h)

In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
p_array = [indexer.token_to_indices(t, None) for t in p_tokens]
h_array = [indexer.token_to_indices(t, None) for t in h_tokens]

In [ ]:
premise = {'elmo': Variable(torch.Tensor(p_array).type(torch.LongTensor))}
hypothesis = {'elmo': Variable(torch.Tensor(h_array).type(torch.LongTensor))}

In [ ]:
outputs = model.forward(premise, hypothesis)

In [ ]:
preds = 100 * outputs['label_probs'].data.numpy()
attn = outputs['p2h_attention'].data.numpy()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

In [ ]:
print('{}\n\n{}'.format(p, h))

In [ ]:
print('Agree: {:.2f}% \nDisagree: {:.2f}% \nUnrelated: {:.2f}%'.
      format(preds[0], preds[1], preds[2]))

In [ ]:
ax = plt.subplots(figsize=(10, 6))
ax = sns.heatmap(np.squeeze(attn), linewidth=0.5)
ax.set_xticklabels(h_tokens)
ax.set_yticklabels(p_tokens, rotation='horizontal')
plt.show()